In [1]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")

In [2]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent, AgentOutputParser
from langchain.agents.chat.base import ChatAgent
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI, VertexAI
from langchain.chat_models import ChatVertexAI, ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [3]:
!export PROJECT_ID="gcp-mira-demo"
project = "gcp-mira-demo"
os.environ["PROJECT_ID"] = project

In [4]:
import config
from config import (VERTEX_LLM_TYPE_BISON_CHAT,
                    VERTEX_LLM_TYPE_BISON_TEXT,
                    OPENAI_LLM_TYPE_GPT3_5,
                    OPENAI_LLM_TYPE_GPT4,
                    COHERE_LLM_TYPE,
                    LLM_BACKEND_ROBOT_USERNAME,
                    LLM_BACKEND_ROBOT_PASSWORD)
from common.utils.token_handler import UserCredentials

config.RULES_ENGINE_BASE_URL = "https://gcp-mira-demo.cloudpssolutions.com/rules-engine/api/v1"
config.auth_client = UserCredentials(LLM_BACKEND_ROBOT_USERNAME,
                              LLM_BACKEND_ROBOT_PASSWORD,
                              "https://gcp-mira-demo.cloudpssolutions.com")

In [5]:
from services.agents.agents import TaskAgent, TaskPlanAgent
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_TEXT)
#casey_plan_agent = TaskPlanAgent(VERTEX_LLM_TYPE_BISON_CHAT)
#casey_plan_agent = TaskPlanAgent(OPENAI_LLM_TYPE_GPT3_5)
casey_plan_agent = TaskPlanAgent(OPENAI_LLM_TYPE_GPT4)
#casey_plan_agent = TaskPlanAgent(COHERE_LLM_TYPE)

In [6]:
prompt = """Send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"""

In [7]:
user_id = "5nJrkPWa3D0yCKA853mD"

In [8]:
tools = casey_plan_agent.get_tools()
agent = casey_plan_agent.load_agent()
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=agent, tools=tools)

chat_history = None
agent_inputs = {
    "input": prompt,
    "chat_history": chat_history
}

In [9]:
#agent_executor.run(agent_inputs)

In [10]:
from services.agents.agent_service import agent_plan, parse_plan

In [11]:
agent_plan("Task", prompt, user_id)

('Task: Send an email to all the Medicaid applicants that are missing income verification asking them to provide a pay stub from their employers.\n\nThought: To accomplish this task, I need to identify the Medicaid applicants with missing income verification, draft an email asking them to provide a pay stub, and finally send the email to everyone identified. I can use the Medicaid CRM tool to find out who is missing income verification, Google Docs to draft the email, and Gmail to send it out.\n\nPlan:\n1. Use [medicaid_crm_retrieve tool] to [retrieve the list of Medicaid applicants missing income verification].\n2. Use [docs_tool] to [draft an email asking the applicants to provide a pay stub from their employers].\n3. Use [gmail_tool] to [send the drafted email to all the applicants missing income verification].',
 <common.models.agent.UserPlan at 0x12f2c3700>)